In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

# Cargue de Queries y parametros

In [2]:
df_kams_ctm = get_fresh_query_result("https://internal-redash.federate.frubana.com/",169591,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_kams_ctm[["customer_id"]] = df_kams_ctm[["customer_id"]].astype(int)

200


In [3]:
df_bench = get_fresh_query_result("https://internal-redash.federate.frubana.com/",175456,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [5]:
query = """
SELECT DISTINCT
split_part(bs.upc, '-',1) as region_code
--,bs.sku_id
--,split_part(bs.upc, ':',4) as IdProd
,bs.addl_product_id as card_id
--,bs.upc
,bcat2.name as categoria
,bcat.name as subcategoria
,        CASE
            WHEN ((bs.active_end_date IS NULL OR bs.active_end_date > CURRENT_TIMESTAMP) AND (bs2.active_end_date IS NULL OR bs2.active_end_date > CURRENT_TIMESTAMP)) THEN 'on'
            ELSE 'off'
        END as status_frida


from postgres_broadleaf_federate."broadleaf.blc_sku" bs
LEFT join postgres_broadleaf_federate."broadleaf.blc_product" bp  on bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku" bs2 on bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
left join postgres_broadleaf_federate."broadleaf.blc_category_xref" bcx on bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and default_reference = 'true' --Relaciones categorias 
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat on bp.default_category_id = bcat.category_id --Nombre subcategoria
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat2 on bcx.category_id = bcat2.category_id --Nombre categoria

where 
bs.archived = 'N'
and bs.sndbx_id is null
and bp.archived = 'N'
and bs.catalog_disc < 0
and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))
"""
df_skus = read_connection_data_warehouse.runQuery(query)

In [6]:
query = """
WITH MaxSubmissionDates AS (
    SELECT
        su.card_id,
        MAX(submit_date.full_date) AS max_submission_date
    FROM dpr_sales.fact_sales s
    LEFT JOIN dpr_shared.dim_date submit_date
        ON s.dim_submitted_date = submit_date.date_id
    LEFT JOIN dpr_shared.dim_stock_unit su
        ON s.dim_product = su.product_id
    WHERE submit_date.full_date >= DATE(GETDATE()) - 10
        AND s.dim_status = 1
    GROUP BY su.card_id
)

SELECT
    s.dim_submitted_date,
    su.card_id,
    CASE WHEN su.source_parent_id = 0 THEN su.source_id ELSE su.source_parent_id END AS parent_id,
    SUM((s.product_price * s.product_quantity_x_step_unit)) / NULLIF(SUM(s.product_quantity_x_step_unit), 0) AS gross_price_no_iva,
    SUM((s.product_price * s.product_quantity_x_step_unit) + s.product_tax_iva) / NULLIF(SUM(s.product_quantity_x_step_unit), 0) AS gross_price
FROM dpr_sales.fact_sales s
LEFT JOIN dpr_shared.dim_site site
    ON s.dim_site = site.site_id
LEFT JOIN dpr_shared.dim_date submit_date
    ON s.dim_submitted_date = submit_date.date_id
LEFT JOIN dpr_shared.dim_stock_unit su
    ON s.dim_product = su.product_id
INNER JOIN MaxSubmissionDates msd
    ON su.card_id = msd.card_id
    AND submit_date.full_date = msd.max_submission_date
GROUP BY 1, 2, 3
"""
df_prices = run_read_dwd_query(query)
df_prices[["gross_price_no_iva","gross_price"]] = df_prices[["gross_price_no_iva","gross_price"]].astype(float)

In [7]:
df_prices = pd.merge(df_prices, df_bench[['extracted_id','benchmark']],  how='left', left_on=['parent_id'], right_on = ['extracted_id'])
df_prices.drop(columns=['extracted_id'], inplace=True)

df_prices = df_prices.groupby(['card_id']).agg({'gross_price_no_iva':np.mean,'gross_price':np.mean,'benchmark':np.mean}).reset_index().copy()

In [8]:
query = """
WITH base_skus AS (
    SELECT DISTINCT
        bo.offer_id,
        bo.offer_value/100.00 AS dct,
        bocsx.customer_segment_id,
        REPLACE(SPLIT_PART(SPLIT_PART(boic.order_item_match_rule, '[',2),']',1),'"','') AS skus
            
    FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_customer_offer_xref"       bcox    ON (bo.offer_id = bcox.offer_id  )
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"       btcofx  ON (btcofx.offer_id = bo.offer_id)
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id AND (boic.archived is NULL OR boic.archived ='N'))
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc

    WHERE   
        (bo.end_date >= CURRENT_dATE or bo.end_date IS NULL)
        AND bo.start_date <= CURRENT_DATE
        ---TIME VALIDATIONS
        -------------------
        AND bo.offer_discount_type = 'PERCENT_OFF'
        AND bo.offer_type = 'ORDER_ITEM'
        AND bo.sndbx_id is NULL
        AND (bo.archived is NULL OR bo.archived='N')
        AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND bo.automatically_added = 'true'
        AND (boic.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND (bocsx.sndbx_tier is NULL OR bocsx.sndbx_tier = 999999)
        --AND bocsx.customer_segment_id IS NOT NULL
        AND bs.site_identifier_value = 'BOG'
        AND (bo.max_uses_per_customer <> 1 OR bo.max_uses_per_customer IS NULL)
    ),
    
    tab_unnest AS (
        SELECT 1 AS n UNION ALL
        SELECT 2 AS n UNION ALL
        SELECT 3 AS n UNION ALL
        SELECT 4 AS n UNION ALL
        SELECT 5 AS n UNION ALL
        SELECT 6 AS n UNION ALL
        SELECT 7 AS n UNION ALL
        SELECT 8 AS n UNION ALL
        SELECT 9 AS n UNION ALL
        SELECT 10 AS n UNION ALL
        SELECT 11 AS n UNION ALL
        SELECT 12 AS n UNION ALL
        SELECT 13 AS n UNION ALL
        SELECT 14 AS n UNION ALL
        SELECT 15 AS n UNION ALL
        SELECT 16 AS n UNION ALL
        SELECT 17 AS n UNION ALL
        SELECT 18 AS n UNION ALL
        SELECT 19 AS n UNION ALL
        SELECT 20 AS n UNION ALL
        SELECT 21 AS n UNION ALL
        SELECT 22 AS n UNION ALL
        SELECT 23 AS n UNION ALL
        SELECT 24 AS n UNION ALL
        SELECT 25 AS n UNION ALL
        SELECT 26 AS n UNION ALL
        SELECT 27 AS n UNION ALL
        SELECT 28 AS n UNION ALL
        SELECT 29 AS n UNION ALL
        SELECT 30 AS n UNION ALL
        SELECT 31 AS n UNION ALL
        SELECT 32 AS n UNION ALL
        SELECT 33 AS n UNION ALL
        SELECT 34 AS n UNION ALL
        SELECT 35 AS n UNION ALL
        SELECT 36 AS n UNION ALL
        SELECT 37 AS n UNION ALL
        SELECT 38 AS n UNION ALL
        SELECT 39 AS n UNION ALL
        SELECT 40 AS n UNION ALL
        SELECT 41 AS n UNION ALL
        SELECT 42 AS n UNION ALL
        SELECT 43 AS n UNION ALL
        SELECT 44 AS n UNION ALL
        SELECT 45 AS n UNION ALL
        SELECT 46 AS n UNION ALL
        SELECT 47 AS n UNION ALL
        SELECT 48 AS n UNION ALL
        SELECT 49 AS n UNION ALL
        SELECT 50 AS n UNION ALL
        SELECT 51 AS n UNION ALL
        SELECT 52 AS n UNION ALL
        SELECT 53 AS n UNION ALL
        SELECT 54 AS n UNION ALL
        SELECT 55 AS n UNION ALL
        SELECT 56 AS n UNION ALL
        SELECT 57 AS n UNION ALL
        SELECT 58 AS n UNION ALL
        SELECT 59 AS n UNION ALL
        SELECT 60 AS n UNION ALL
        SELECT 61 AS n UNION ALL
        SELECT 62 AS n UNION ALL
        SELECT 63 AS n UNION ALL
        SELECT 64 AS n UNION ALL
        SELECT 65 AS n UNION ALL
        SELECT 66 AS n UNION ALL
        SELECT 67 AS n UNION ALL
        SELECT 68 AS n UNION ALL
        SELECT 69 AS n UNION ALL
        SELECT 70 AS n UNION ALL
        SELECT 71 AS n UNION ALL
        SELECT 72 AS n UNION ALL
        SELECT 73 AS n UNION ALL
        SELECT 74 AS n UNION ALL
        SELECT 75 AS n UNION ALL
        SELECT 76 AS n UNION ALL
        SELECT 77 AS n UNION ALL
        SELECT 78 AS n UNION ALL
        SELECT 79 AS n UNION ALL
        SELECT 80 AS n UNION ALL
        SELECT 81 AS n UNION ALL
        SELECT 82 AS n UNION ALL
        SELECT 83 AS n UNION ALL
        SELECT 84 AS n UNION ALL
        SELECT 85 AS n UNION ALL
        SELECT 86 AS n UNION ALL
        SELECT 87 AS n UNION ALL
        SELECT 88 AS n UNION ALL
        SELECT 89 AS n UNION ALL
        SELECT 90 AS n UNION ALL
        SELECT 91 AS n UNION ALL
        SELECT 92 AS n UNION ALL
        SELECT 93 AS n UNION ALL
        SELECT 94 AS n UNION ALL
        SELECT 95 AS n UNION ALL
        SELECT 96 AS n UNION ALL
        SELECT 97 AS n UNION ALL
        SELECT 98 AS n UNION ALL
        SELECT 99 AS n UNION ALL
        SELECT 100 AS n UNION ALL
        SELECT 101 AS n UNION ALL
        SELECT 102 AS n UNION ALL
        SELECT 103 AS n UNION ALL
        SELECT 104 AS n UNION ALL
        SELECT 105 AS n UNION ALL
        SELECT 106 AS n UNION ALL
        SELECT 107 AS n UNION ALL
        SELECT 108 AS n UNION ALL
        SELECT 109 AS n UNION ALL
        SELECT 110 AS n UNION ALL
        SELECT 111 AS n UNION ALL
        SELECT 112 AS n UNION ALL
        SELECT 113 AS n UNION ALL
        SELECT 114 AS n UNION ALL
        SELECT 115 AS n UNION ALL
        SELECT 116 AS n UNION ALL
        SELECT 117 AS n UNION ALL
        SELECT 118 AS n UNION ALL
        SELECT 119 AS n UNION ALL
        SELECT 120 AS n
    ),
     
    info_skus AS (  
    SELECT 
        bs.offer_id,
        bs.dct,
        bs.customer_segment_id,
        TRIM(SPLIT_PART(bs.skus, ',', tab_unnest.n)) AS sku
        
    FROM tab_unnest
    INNER JOIN base_skus bs ON tab_unnest.n <= REGEXP_COUNT(bs.skus, ',') + 1
    )
    
SELECT DISTINCT
    s.site_identifier_value as ciudad,
    --bcs.external_identifier::int AS customer_id,
    bs.addl_product_id AS card_id,
    MAX(bo.dct) AS dct
    
FROM info_skus bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bo.sku
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id AND bo.customer_segment_id IS NOT NULL AND bcs.external_identifier ~ '^[0-9]+$'
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON s.site_id = bs.catalog_disc

WHERE bo.customer_segment_id IS NULL
GROUP BY 1,2--,3
"""
df_no_filt_offers = read_connection_data_warehouse.runQuery(query)
df_no_filt_offers[["dct"]] = df_no_filt_offers[["dct"]].astype(float)

In [9]:
query = """
SELECT 
    s.identifier_value AS region_code,
    su.card_id,
    AVG(fbi.margin_mtd) AS margin
FROM dpr_cross_business.fact_cross_business_insights fbi
INNER JOIN dpr_shared.dim_stock_unit su ON su.stock_unit_id = fbi.dim_stock_unit
INNER JOIN dpr_shared.dim_site s ON s.site_id = fbi.dim_site
WHERE (dim_date,dim_stock_unit) IN (SELECT MAX(dim_date), dim_stock_unit FROM dpr_cross_business.fact_cross_business_insights WHERE margin_mtd <> 0 GROUP BY 2)
GROUP BY 1,2
"""
df_margen = run_read_dwd_query(query)
df_margen[["margin"]] = df_margen[["margin"]].astype(float)

In [10]:
query = """
SELECT 
s.identifier_value AS city,
DATE(fs.order_submitted_date) AS submit_date,
fs.order_id,
dc.source_id AS customer_id,
cat.parent_description as category,
cat.description AS subcategory,
dp.card_id,
dp.card_description AS product_name,
fs.product_quantity_x_step_unit AS cant,
CASE
    WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
    WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
    WHEN s.identifier_value IN ('CMX') THEN fs.gmv_pxq_local/19.65
    ELSE fs.gmv_pxq_local
END AS gmv_usd,
CASE
    WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
    WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
    WHEN s.identifier_value IN ('CMX') THEN fsd.product_discount/19.65
    ELSE fsd.product_discount
END AS discount_applied
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
WHERE fs.gmv_enabled = TRUE
AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
AND fs.fb_order_status_id  IN (1,6,7,8)
AND fs.is_deleted = FALSE
AND fs.dim_status = 1
AND dp.is_slot = 'false'
AND s.identifier_value IN ('BOG','BAQ','MDE','CMX','SPO','BHZ','CWB','VCP')
AND dc.source_id IN {clientes}
""".format(clientes=tuple(df_kams_ctm.customer_id.unique()))

data_ventas_aux = run_read_dwd_query(query)
data_ventas_aux[["gmv_usd", "cant","discount_applied"]] = data_ventas_aux[["gmv_usd", "cant","discount_applied"]].astype(float)
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [11]:
df = pd.merge(data_ventas_aux, df_margen[['card_id','margin']],  how='left', left_on=['card_id'], right_on = ['card_id'])

In [12]:
df['discount_applied'] = df['discount_applied'].fillna(value=0)
df['margin'] = df['margin'].fillna(value=0.1)

df['cash_margin'] = df.gmv_usd * df.margin
df['net_cash_margin'] = df.cash_margin - df.discount_applied

df_products = df.copy()

In [13]:
df = df.loc[df.card_id.isin(df_skus.card_id.unique())].groupby(by=['city','customer_id','submit_date','category','subcategory']).agg({'gmv_usd':'sum',
                                                                                                                                    'cant':'sum',
                                                                                                                                    'discount_applied':'sum',
                                                                                                                                    'cash_margin':'sum',
                                                                                                                                    'net_cash_margin': 'sum'}).reset_index()

# Generacion de df con info by product by client

In [14]:
df = df.sort_values('submit_date',ignore_index=True)

#fix for id problems and taking into account only dates
df['order_id']= df.submit_date.apply(lambda x:x.toordinal()) 

#Days of difference between orders for customer and for cat
df['days_bet'] = (df.submit_date -df.groupby('customer_id').submit_date.transform('min')).dt.days

In [15]:
#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df['res'] = df.groupby(by=['customer_id', 'category', 'subcategory'])['days_bet'].diff()
df_grouped = df.groupby(by=['city', 'customer_id', 'category', 'subcategory']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                                    'order_id': [lambda x: x.nunique()], 
                                                                                    'gmv_usd': [np.sum, np.mean],
                                                                                    'discount_applied': [np.sum, np.mean],
                                                                                    'cash_margin': [np.sum, np.mean],
                                                                                    'net_cash_margin': [np.sum, np.mean],
                                                                                    'cant': [np.sum, np.mean],
                                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                                    }).reset_index()

In [16]:
#Finally we define everything in a df2 for further processing
df2 = df_grouped.copy()

df2.columns = ['city', 'customer_id', 'category', 'subcategory', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'gmv_cat_usd','aov_cat_usd', 'all_dct_usd', 'avg_dct_usd','all_cash_margin','aov_cash_margin','all_net_cash_margin','aov_net_cash_margin','all_cant_cat','avg_cant_cat', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [17]:
df2["churned"] = [0 if ((x[22]) - (x[6]+x[8]*2) <= 0) else 1 for x in df2.values]

df2['churned_date'] = df2.apply(lambda x: (x[20] + datetime.timedelta(days=int(x[6]) + int(x[8]) * 2)).strftime('%Y-%m-%d') if not (pd.isnull(x[6]) or pd.isnull(x[8])) else None, axis=1)
df2['churned_date'] = pd.to_datetime(df2['churned_date'], errors='coerce')

In [18]:
df2.head()

,city,customer_id,category,subcategory,min_diff_days,max_diff_days,avg_diff_days,median_diff_days,stdev_diff_days,num_orders,...,all_net_cash_margin,aov_net_cash_margin,all_cant_cat,avg_cant_cat,last_order_date,diff_from_last_to_first_order,days_since_last_order,first_order_date,churned,churned_date
0,BAQ,-253219,Abarrotes,Aceites,7.0,148.0,25.692308,14.0,37.840421,14,...,93.209620,6.657830,23.00,1.642857,2023-10-21,334,16,2022-11-21,0,2024-01-28
1,BAQ,-253219,Abarrotes,Arroz,1.0,92.0,7.880000,7.0,10.460833,126,...,51.178535,0.406179,689.04,5.468571,2023-11-04,985,2,2021-02-22,0,2023-12-01
2,BAQ,-253219,Abarrotes,Azúcar & endulzantes,1.0,40.0,7.530973,7.0,6.235083,114,...,650.518961,5.706307,523.00,4.587719,2023-11-01,851,5,2021-07-03,0,2023-11-20
3,BAQ,-253219,Abarrotes,Granos,2.0,35.0,8.968254,7.0,5.239331,64,...,22.115502,0.345555,827.02,12.922187,2023-11-04,565,2,2022-04-18,0,2023-11-22
4,BAQ,-253219,Abarrotes,Harinas & mezclas,2.0,28.0,9.314286,8.0,4.778385,106,...,78.261704,0.738318,145.00,1.367925,2023-11-04,978,2,2021-03-01,0,2023-11-21


# Merge all info 

In [19]:
df_products.head()

,city,submit_date,order_id,customer_id,category,subcategory,card_id,product_name,cant,gmv_usd,discount_applied,margin,cash_margin,net_cash_margin
0,CMX,2020-05-31,431009,-263212,Frutas & verduras,Verduras,-201497,Papa Cambray Blanca,3.0,2.748092,0.0,0.0435,0.119542,0.119542
1,CMX,2020-05-31,431009,-263212,Frutas & verduras,Verduras,-201420,Tomate Verde,1.0,1.012723,0.0,0.1515,0.153427,0.153427
2,CMX,2020-05-31,432041,-256946,Frutas & verduras,Frutas,-201381,Naranja Valencia,20.4,17.129771,0.0,0.5828,9.983231,9.983231
3,BOG,2020-05-31,431465,-200025,Frutas & verduras,Verduras,-200060,Pepino Cohombro,4.0,1.705508,0.0,0.0749,0.127743,0.127743
4,BOG,2020-05-31,478316,-247604,Frutas & verduras,Frutas jugo,-200734,Lulo Pequeño,3.0,2.786679,0.0,0.1185,0.330221,0.330221


In [20]:
df_products2 = pd.merge(df_products, df_skus.loc[df_skus.status_frida == 'on',['card_id']],  how='inner', left_on=['card_id'], right_on = ['card_id'])

In [21]:
df_products2 = pd.merge(df_products2, df_prices,  how='inner', left_on=['card_id'], right_on = ['card_id'])
df_products2 = pd.merge(df_products2, df_no_filt_offers[['card_id','dct']],  how='left', left_on=['card_id'], right_on = ['card_id'])

In [22]:
df_products2['dct'].fillna(0, inplace=True)

In [23]:
df_products2['net_price'] = (df_products2.gross_price_no_iva * (1-df_products2.dct)) + (df_products2.gross_price - df_products2.gross_price_no_iva)

df_products2['gpi'] = 100.00 * df_products2.gross_price / df_products2.benchmark
df_products2['npi'] = 100.00 * df_products2.net_price / df_products2.benchmark

# Merge Products AND Customers

In [24]:
# Get the current date
current_date = datetime.datetime.now()

# Calculate the start and end dates of the current week
start_of_week = current_date - timedelta(days=current_date.weekday())
end_of_week = start_of_week + timedelta(days=6)
current_date2 = datetime.datetime.now().strftime("%d-%m-%Y")

In [25]:
# Define the current date, last month, and the month before last
last_month = current_date - timedelta(days=7)
month_before_last = current_date - timedelta(days=90)  # Two months before

In [26]:
df2 = df2.loc[(df2.num_orders>3) & (df2.churned_date >= month_before_last) & (df2.churned_date <= last_month)]

In [27]:
from sklearn.model_selection import train_test_split

def create_groups(offer):
    customers = offer.customer_id.unique()
    
    alpha = 0.01
    aux = 0
    
    while True:
        customer_control, customer_test = train_test_split(customers, test_size=0.50)
        
        avg_gmv_control = offer[offer.customer_id.isin(customer_control)]['aov_cat_usd'].mean()
        avg_gmv_test = offer[offer.customer_id.isin(customer_test)]['aov_cat_usd'].mean()
        
        # Check if both average GMV and average products meet the criteria
        if (
            abs(1 - (avg_gmv_control / avg_gmv_test)) <= alpha
        ):
            break
        
        aux += 1
        
        if aux == 1000:
            alpha += 0.01
            aux = 0
    print(alpha)
    return customer_control.tolist(), customer_test.tolist()

In [28]:
result_rows = []

for city in df2.city.unique():
    df_aux_1 = df2.loc[df2.city == city].copy()
    for subcat in df_aux_1.subcategory.unique():
        df_aux_2 = df_aux_1.loc[df_aux_1.subcategory == subcat]
        if df_aux_2.customer_id.nunique() < 4:
            print(f'{subcat} MUY PEQUEÑA')
            pass
        else:
            control, piloto = create_groups(df_aux_2.groupby(['customer_id']).aov_cat_usd.sum().reset_index())
            for customer_id in control:
                result_rows.append([city, subcat, customer_id, 'control'])
            for customer_id in piloto:
                result_rows.append([city, subcat, customer_id, 'piloto'])
            #break
    #break
result_df = pd.DataFrame(result_rows, columns=['city', 'subcat','customer_id', 'group'])

Embutidos MUY PEQUEÑA
Alimentos congelados MUY PEQUEÑA
Quesos MUY PEQUEÑA
0.21000000000000005
Arroz MUY PEQUEÑA
Cremas MUY PEQUEÑA
Res congelado MUY PEQUEÑA
Pescados & mariscos congelado MUY PEQUEÑA
0.36000000000000015
Desinfectantes & sanitizantes MUY PEQUEÑA
0.46000000000000024
Salsas MUY PEQUEÑA
Harinas & mezclas MUY PEQUEÑA
Vasos MUY PEQUEÑA
Mantequillas MUY PEQUEÑA
Cubiertos & pitillos MUY PEQUEÑA
Leches MUY PEQUEÑA
Tubérculos MUY PEQUEÑA
Res fresco MUY PEQUEÑA
Huevos MUY PEQUEÑA
Implementos de aseo MUY PEQUEÑA
Panadería & repostería MUY PEQUEÑA
Papel & toallas MUY PEQUEÑA
Sal & sazonadores MUY PEQUEÑA
Derivados lácteos MUY PEQUEÑA
Verduras MUY PEQUEÑA
0.01
Peixes e frutos do mar MUY PEQUEÑA
0.07
0.01
0.01
0.01
0.01
0.01
Leite MUY PEQUEÑA
Desinfetantes e sanitizantes MUY PEQUEÑA
0.2700000000000001
0.05
0.02
0.2700000000000001
0.02
0.09999999999999999
0.3000000000000001
0.01
Massas e molhos MUY PEQUEÑA
Temperos MUY PEQUEÑA
0.04
0.02
0.47000000000000025
Papéis MUY PEQUEÑA
0.02
0.240

In [29]:
df_products3 = df_products2.loc[(df_products2.gpi - df_products2.margin * 100.00 * 0.9 < 100) & (df_products2.npi > 94)].copy()

In [30]:
df_offers = pd.merge(df_products3, result_df,  how='inner', left_on=['city','subcategory','customer_id'], right_on = ['city','subcat','customer_id'])

In [31]:
df_offers = df_offers[['city','customer_id', 'category',
       'subcategory', 'card_id', 'product_name','margin','gross_price_no_iva', 'gross_price', 'benchmark', 'dct', 'net_price',
       'gpi', 'npi', 'group']].drop_duplicates()

In [32]:
df_offers.to_excel(f"{current_date2}_KAM_analysis_GROUPS_OK_SUBCAT.xlsx")

In [33]:
df_dct = df_offers.loc[df_offers.group == 'piloto'].copy()

In [34]:
df_dct

,city,customer_id,category,subcategory,card_id,product_name,margin,gross_price_no_iva,gross_price,benchmark,dct,net_price,gpi,npi,group
295,CMX,-265901,Frutas & verduras,Frutas,-201381,Naranja Valencia,0.5828,17.217000,17.217000,16.00,0.0,17.217000,107.606250,107.606250,piloto
297,CMX,-265901,Frutas & verduras,Frutas,-201409,Aguacate Hass,0.2724,54.090667,54.090667,49.00,0.0,54.090667,110.389116,110.389116,piloto
306,CMX,-265901,Frutas & verduras,Frutas,-201429,Limón con semilla,0.2323,30.595050,30.595050,29.50,0.0,30.595050,103.712034,103.712034,piloto
309,CMX,-265901,Frutas & verduras,Frutas,-201466,Manzana Roja,0.1809,52.500000,52.500000,50.00,0.0,52.500000,105.000000,105.000000,piloto
310,CMX,-265901,Frutas & verduras,Frutas,-201509,Naranja - Jugo,0.6008,14.399400,14.399400,13.00,0.0,14.399400,110.764615,110.764615,piloto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101342,VCP,344571188,Mercearia,Massas e molhos,457388,Molho de Tomate Tradicional Heinz Sachê 300g,0.2832,2.890000,2.890000,2.99,0.0,2.890000,96.655518,96.655518,piloto
101345,CWB,137244943,"Carnes, aves e peixes",Carne vermelha,396643,Carne Seca Bovina de Dianteiro - Mineiro,0.1821,35.700000,35.700000,36.90,0.0,35.700000,96.747967,96.747967,piloto
101388,BHZ,67588215,"Carnes, aves e peixes",Frango,409531,Coxa e Sobrecoxa de Frango Desossada - Avivar,0.1962,13.160000,13.160000,12.90,0.0,13.160000,102.015504,102.015504,piloto
101399,BHZ,64526650,Mercearia,Farinhas e misturas,451670,Farinha de Trigo Premium Rosa Branca 5kg,0.3011,30.310000,30.310000,24.90,0.0,30.310000,121.726908,121.726908,piloto


In [35]:
condition_city = df_dct.city.isin(['CMX', 'BAQ', 'BOG', 'MDE'])
# Calculate the 'dct' column conditionally
df_dct['dct_final'] = np.where(condition_city,  # Apply the condition to df_dct
                         np.round((1 - (df_dct.benchmark * 0.94 / df_dct.gross_price)) * 100.00, 2),
                         np.round((1 - (df_dct.benchmark * 0.94 / df_dct.gross_price)) * 100.00, 2))

In [36]:
#df_dct['dct'] = np.round((1 - (df_dct.benchmark * 0.99 / df_dct.gross_price)) * 100.00,2)
df_dct['proportion_dct'] = np.round(df_dct.dct / (df_dct.dct_final/100.00),2)  

In [37]:
df_dct[['proportion_dct']] = df_dct[['proportion_dct']].fillna(0)

In [38]:
df_dct.proportion_dct.sort_values()

295      0.00
80347    0.00
80405    0.00
80407    0.00
80408    0.00
         ... 
21765    0.96
21480    0.96
22370    0.96
20864    0.96
22474    0.96
Name: proportion_dct, Length: 3596, dtype: float64

In [39]:
df_dct = df_dct.loc[(df_dct.proportion_dct < 1) & (df_dct.dct_final > 0.005)]

In [40]:
df_dct

,city,customer_id,category,subcategory,card_id,product_name,margin,gross_price_no_iva,gross_price,benchmark,dct,net_price,gpi,npi,group,dct_final,proportion_dct
295,CMX,-265901,Frutas & verduras,Frutas,-201381,Naranja Valencia,0.5828,17.217000,17.217000,16.00,0.0,17.217000,107.606250,107.606250,piloto,12.64,0.0
297,CMX,-265901,Frutas & verduras,Frutas,-201409,Aguacate Hass,0.2724,54.090667,54.090667,49.00,0.0,54.090667,110.389116,110.389116,piloto,14.85,0.0
306,CMX,-265901,Frutas & verduras,Frutas,-201429,Limón con semilla,0.2323,30.595050,30.595050,29.50,0.0,30.595050,103.712034,103.712034,piloto,9.36,0.0
309,CMX,-265901,Frutas & verduras,Frutas,-201466,Manzana Roja,0.1809,52.500000,52.500000,50.00,0.0,52.500000,105.000000,105.000000,piloto,10.48,0.0
310,CMX,-265901,Frutas & verduras,Frutas,-201509,Naranja - Jugo,0.6008,14.399400,14.399400,13.00,0.0,14.399400,110.764615,110.764615,piloto,15.14,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101342,VCP,344571188,Mercearia,Massas e molhos,457388,Molho de Tomate Tradicional Heinz Sachê 300g,0.2832,2.890000,2.890000,2.99,0.0,2.890000,96.655518,96.655518,piloto,2.75,0.0
101345,CWB,137244943,"Carnes, aves e peixes",Carne vermelha,396643,Carne Seca Bovina de Dianteiro - Mineiro,0.1821,35.700000,35.700000,36.90,0.0,35.700000,96.747967,96.747967,piloto,2.84,0.0
101388,BHZ,67588215,"Carnes, aves e peixes",Frango,409531,Coxa e Sobrecoxa de Frango Desossada - Avivar,0.1962,13.160000,13.160000,12.90,0.0,13.160000,102.015504,102.015504,piloto,7.86,0.0
101399,BHZ,64526650,Mercearia,Farinhas e misturas,451670,Farinha de Trigo Premium Rosa Branca 5kg,0.3011,30.310000,30.310000,24.90,0.0,30.310000,121.726908,121.726908,piloto,22.78,0.0


In [41]:
df_dct["offer_name"] = (
    "KAM_PTOOL_EXP.DCT_" +
    df_dct["city"] + "_" +
    df_dct["product_name"].str.replace(",", ".") + "_" + current_date2 + "_" +
    df_dct["dct"].astype(str) + "%_HDCT_" + 
    df_dct["proportion_dct"].astype(str)
)

In [42]:
def segments(offer, cityy):
        import requests
        import json
        
        global_segment_list = [] #//for procesing
        for i in offer.customer_id.unique():
                segment = {'name':str(i), 'customersIds': [int(i)],"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(7))).strftime('%Y-%m-%d')}
                global_segment_list.append(segment) 
        
        url = f"https://{cityy}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

        payload = json.dumps(global_segment_list)


        headers = {
        'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
        'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)
        
        sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
        ssgtt = pd.DataFrame(sgt, columns=['segment_id', 'customer_id'])
        ssgtt[["customer_id"]] = ssgtt[["customer_id"]].astype(int)
        
        return ssgtt                

In [43]:
def csv_files(offer,segment):
    csv_file = {} 
    csv_file['offer_name'] = offer.offer_name
    csv_file['offer_description'] = None
    csv_file['discount'] = offer.dct_final
    csv_file['automatically_consider_offer'] = "true"
    csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=0)).strftime('%d-%m-%Y')
    csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(7))).strftime('%d-%m-%Y')
    csv_file['max_uses_per_order'] = 0
    csv_file['max_uses_per_customer'] = 2
    csv_file['customer_segment_id'] = [segment.loc[segment.customer_id == customer].segment_id.unique()[0] for customer in offer.customer_id]

    sku_ids = []
    for sku in offer.card_id:
        try:
            sku_id = frida_products.loc[frida_products.id_tarjeta == sku, "skus"].values[0]
        except IndexError:
            sku_id = None  # Assign a null value if IndexError occurs
        sku_ids.append(sku_id)
    
    csv_file['sku_id'] = sku_ids
    
    df = pd.DataFrame(csv_file)
    
    # Drop rows with null values in the 'sku_id' column
    df = df.dropna(subset=['sku_id'])
    return df

In [44]:
for city in df_dct.city.unique():
    print(city)
    
    segment = segments(df_dct.loc[df_dct.city == city],city)
    
    city_csv = csv_files(df_dct.loc[df_dct.city == city],segment)
    city_csv = city_csv.drop_duplicates()

    # Concatenate the values in the 'customer_segment_id' column with "#"
    city_csv['customer_segment_id'] = city_csv.groupby('sku_id')['customer_segment_id'].transform(lambda x: '#'.join(x.astype(str)))

    # Drop duplicate rows
    city_csv = city_csv.drop_duplicates(subset=['sku_id'])

    # Reset index if needed
    city_csv = city_csv.reset_index(drop=True)

    city_csv = city_csv.loc[city_csv.discount > 0].copy()

    city_csv.to_csv(f"{city}_KAM_EXP.DCT{current_date2}.csv", index = False)

CMX
BOG
BAQ
SPO
BHZ
VCP
CWB
MDE
